<a href="https://colab.research.google.com/github/aabitokh/home_repo/blob/develop/ntb_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# imports

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 99.9 MB/s eta 0:00:00


In [2]:
import pandas as pd 
import numpy as np 

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score

import torch
import transformers
from tqdm import notebook
from torch import tensor


from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import re

In [3]:
DATA_PATH = 'toxic_comments.csv'
#DATA_PATH = '/datasets/toxic_comments.csv'

In [4]:
def clear_text(text):
    return ' '.join(re.sub(r'[^a-zA-Z\' ]', ' ', text).split())

In [5]:
#проверка на GPU
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
!nvidia-smi

Thu Mar 16 22:23:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    25W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# first look 

In [7]:
import csv

In [22]:
try:
    df = pd.read_csv(DATA_PATH)
except pd.errors.ParserError:
    df = pd.read_csv(DATA_PATH, encoding='utf-8', engine='python', delimiter='\t')


In [23]:
df=df.sample(1292)

In [24]:
df.sample(10)

,Unnamed: 0,text,toxic
26715,26750,"""\n\n I hope this brightens your day \n Hello ...",0
60281,60347,"""\nWhich just proves my point. Hasteur warned ...",0
13924,13939,fuck all this shit in to dickmania,1
119363,119468,"""\n\n October Baseball WikiProject Newsletter ...",0
107693,107790,"There is already a process for this, at WP:CFD...",0
74629,74703,You to. 82.132.218.163,0
157736,157895,"Support Based on what PatGallacher asserts, I ...",0
26087,26121,This article does not meet wikipedia standard ...,0
107359,107456,"""\n\nStop interacting with Someone65. End of s...",0
15542,15559,"You did abuse your position. As I said above, ...",0


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1292 entries, 34648 to 118193
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1292 non-null   int64 
 1   text        1292 non-null   object
 2   toxic       1292 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 40.4+ KB


# data preprocessing

In [26]:
df = df.drop(['Unnamed: 0'], axis = 1) 

In [27]:
df

,text,toxic
34648,Segmnet in Riding Giants film \n\nCan we add t...,0
1653,Week 6 of WIKISOO\nHi ! Just checking in as we...,0
40279,"""::Hi, and thanks for coming to the talkpage. ...",0
146988,"""\n\n feel sorry for you \n\nPoor you're a big...",0
130409,"This article is a vile, pathetic(not unlike th...",1
...,...,...
14287,Wowwww. \n\nYou're really fucking ignorant. -,1
32760,US County Map Projection,0
65238,Wikipedia:Articles for deletion/Proteus (WAM-V...,0
75339,"UPDATE: Charmingly, just blindly reverted fou...",0


In [28]:
df.toxic.value_counts(normalize=True)

0    0.897059
1    0.102941
Name: toxic, dtype: float64

In [29]:
df['text'] = df['text'].apply(lambda x: clear_text(x))

# BERT

In [30]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer, 'distilbert-base-uncased')

#model_class, tokenizer_class, pretrained_weights = (transformers.BertModel, transformers.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [31]:
tokenized = df['text'].apply(lambda x: \
                             tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=500))

In [32]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
        
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

In [33]:
padded = torch.LongTensor(padded).to(DEVICE)
attention_mask = torch.LongTensor(attention_mask).to(DEVICE)

In [34]:
!nvidia-smi

Thu Mar 16 22:28:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W /  70W |    591MiB / 15360MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [35]:
model.to(DEVICE)

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0): TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(i

In [46]:
batch_size = 100
embeddings = []

for i in notebook.tqdm(range((padded.shape[0] // batch_size)+1)):
    batch = padded[batch_size*i:batch_size*(i+1)]
    attention_mask_batch = attention_mask[batch_size*i:batch_size*(i+1)]

    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)

    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/13 [00:00<?, ?it/s]

In [47]:
embeddings[-1].shape

(92, 768)

In [48]:
features = np.concatenate(embeddings)
features

array([[ 2.39976980e-02, -1.02140360e-01, -2.06484184e-01, ...,
        -5.22625670e-02,  3.76746535e-01,  2.52643049e-01],
       [ 1.57956973e-01, -3.41452695e-02,  5.03429472e-02, ...,
         4.60668579e-02,  5.50808787e-01,  1.48525804e-01],
       [-6.34522513e-02, -6.72269939e-03,  1.75995201e-01, ...,
        -5.23957945e-02,  5.80466032e-01,  1.62306100e-01],
       ...,
       [ 1.13001950e-01, -1.18766814e-01,  1.08969398e-01, ...,
         5.68884686e-02,  4.87349987e-01,  4.28130597e-01],
       [ 1.18578836e-01,  1.35441095e-01,  9.61522907e-02, ...,
         1.29465267e-01,  4.29796278e-01,  3.49413633e-01],
       [ 3.97729278e-02,  1.55092161e-02,  6.35367876e-04, ...,
         7.79214948e-02,  6.67714834e-01,  1.90990329e-01]], dtype=float32)

In [49]:
len(features)

1292

In [50]:
from sklearn.linear_model import LogisticRegression

In [51]:
lr = LogisticRegression()

lr.fit(features, df['toxic'])

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [52]:
pred = lr.predict(features)

In [53]:
from sklearn.metrics import accuracy_score

In [54]:
accuracy_score(pred, df['toxic'])

0.9752321981424149